# 30 · Embedding-Based Code Matching Demo  
_Last updated 2025-05-03_

**Goal**

1. **Part A** – map LLM-generated *economic variable* nodes onto **JEL** codes.  
2. **Part B** – map LLM-generated *product descriptions* onto **HS-6** codes.

Both parts share the same recipe:

1. Build embeddings (`OpenAI text-embedding-3-large`).  
2. Compute cosine-similarity matrix.  
3. Pick **argmax** (or top-k) per node.

In production you embed the *canonical* code vocabulary **once**, cache it, and
batch-embed new nodes as they arrive.



### If you use this code, please cite the paper(s): 

- Garg, P. and Fetzer, T., 2025. **Causal claims in economics**. arXiv preprint arXiv:2501.06873.
- Fetzer, T., Lambert, P.J., Feld, B. and Garg, P., 2024. **AI-generated production networks: Measurement and applications to global trade**.


## API key + install

Looks for `OPENAI_API_KEY`, else reads `key/openai_key.txt`  
(same pattern as earlier notebooks).


In [1]:
# %pip -q install --upgrade openai python-dotenv langchain_openai scikit-learn pandas numpy

import os, pathlib, numpy as np, pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings

# key file
key_file = pathlib.Path("key/openai_key.txt")
if os.getenv("OPENAI_API_KEY") is None and key_file.exists():
    os.environ["OPENAI_API_KEY"] = key_file.read_text().strip()

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Provide OPENAI_API_KEY or create key/openai_key.txt")

client = OpenAI()


### 1 · Toy nodes (6) and a mini JEL slice (8 codes)

In practice you’d load **all 800+ JEL codes**; here we keep eight for speed.


In [2]:
nodes = [
    "access to microcredit",
    "politics in money",
    "campaign finance",
    "household expenditures",
    "export growth",
    "inflation targeting regime",
    "foreign direct investment inflows",
    "labor force participation of women"
]

jel = pd.DataFrame({
    "code": ["G21", "X99", "D12", "F14", "E31", "F21", "J22", "E52", "O17"],
    "description": [
        "Banks; Microfinance",
        "Microcredit Access",
        "Consumer Economics: Empirical Analysis",
        "Empirical Studies of Trade",
        "Price Level; Inflation; Deflation",
        "International Investment; Long-Term Capital Movements",
        "Time Allocation and Labor Supply",
        "Monetary Policy; Central Banking",
        "Formal and Informal Sectors; Shadow Economy; Institutional Arrangements"
    ]
})


### 2 · Build embeddings & find best match


In [3]:
embedder = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=3072)
# king - man + woman = queen
node_emb = embedder.embed_documents(nodes)                # shape (6,384)
jel_emb  = embedder.embed_documents(jel.description.tolist())  # (8,384)

sim = cosine_similarity(jel_emb, node_emb)  # (8 jel description ×6 nodes)
best_idx = sim.argmax(axis=0)

jel_best = jel.iloc[best_idx].reset_index(drop=True)
df_jel_map = pd.DataFrame({
    "node": nodes,
    "jel_code": jel_best.code,
    "jel_desc": jel_best.description,
    "similarity": sim[best_idx, range(sim.shape[1])]
})
df_jel_map.style.background_gradient(cmap="Greens", subset=["similarity"])


,node,jel_code,jel_desc,similarity
0,access to microcredit,X99,Microcredit Access,0.860630
1,politics in money,E52,Monetary Policy; Central Banking,0.417038
2,campaign finance,G21,Banks; Microfinance,0.287730
3,household expenditures,D12,Consumer Economics: Empirical Analysis,0.450936
4,export growth,D12,Consumer Economics: Empirical Analysis,0.310889
5,inflation targeting regime,E31,Price Level; Inflation; Deflation,0.423500
6,foreign direct investment inflows,F21,International Investment; Long-Term Capital Movements,0.461549
7,labor force participation of women,J22,Time Allocation and Labor Supply,0.416097


In [4]:
node_emb

[[-0.02861475758254528,
  0.06346308439970016,
  0.0020111980848014355,
  -0.03234495595097542,
  -0.017490480095148087,
  -0.001667190925218165,
  0.03297494351863861,
  0.052189603447914124,
  -0.007298752665519714,
  0.026078222319483757,
  -0.004936294164508581,
  0.005197408143430948,
  0.0044886707328259945,
  -0.046851277351379395,
  -0.019513076171278954,
  -0.03975561261177063,
  0.024751931428909302,
  -0.055604804307222366,
  0.0006963034975342453,
  0.026310324668884277,
  -0.01630510576069355,
  -0.03324020281434059,
  -0.0024432791396975517,
  -0.016752729192376137,
  -0.0071827019564807415,
  0.013975805602967739,
  -0.035577792674303055,
  0.006764091085642576,
  -0.0376998595893383,
  -0.010104689747095108,
  0.022298289462924004,
  0.0002515292726457119,
  -0.04930491745471954,
  -0.043867118656635284,
  0.017357850447297096,
  -0.029742104932665825,
  -0.01060204952955246,
  -0.05073068290948868,
  0.0036535209510475397,
  -0.04021981358528137,
  0.018335990607738495

### 3 · (Optional) top-k matches

Inspect top-3 alternatives for one node.


In [5]:
k = 3
target_col = 2           # index for "export growth"
top_idx = np.argsort(sim[:, target_col])[-k:][::-1]
pd.concat([
    jel.iloc[top_idx].reset_index(drop=True),
    pd.Series(sim[top_idx, target_col], name="similarity")
], axis=1)


,code,description,similarity
0,G21,Banks; Microfinance,0.287730
1,E52,Monetary Policy; Central Banking,0.273886
2,X99,Microcredit Access,0.250045


### 4 · Mock LLM product list (10 items) & HS-6 slice (20 rows)

We reuse solar-panel inputs but add a few extras to show variety.


In [6]:
products = [
    "Ultra-clear tempered glass sheets",
    "Silver conductive paste",
    "EVA encapsulant film",
    "Backsheet made of PVF",
    "Aluminium alloy frame sections",
    "Junction boxes with bypass diodes",
    "Poly-silicon wafers",
    "Copper ribbon inter-connects",
    "Phosphorus diffusion chemicals",
    "Anti-reflective coating materials"
]

hs6_sample = pd.DataFrame({
    "code": [
        "700719","710692","392020","392099","760429",
        "853710","381800","740911","280461","320990",
        "854449","850760","392010","392113","381590",
        "760611","282720","340319","850780","850790"
    ],
    "description": [
        "Safety glass, tempered or laminated",
        "Silver, in semi-manufactured forms",
        "Plates, sheets, film of polyethylene",
        "Plates, sheets, film of plastics nes",
        "Aluminium alloy hollow profiles",
        "Switchgear for voltage ≤ 1000 V",
        "Silicon doped for electronic use",
        "Copper plates and sheets, >0.15 mm",
        "Silicon ≥ 99.99 % purity",
        "Pigments and preparations nes",
        "Winding wire of copper",
        "Lithium-ion accumulators",
        "Polyethylene sheets < 0.125 mm",
        "EVA (ethylene-vinyl acetate) sheets",
        "Catalyst & accelerator preparations",
        "Aluminium plates > 0.2 mm",
        "Calcium hydrogen-phosphate",
        "Lubricating preparations nes",
        "Other accumulators",
        "Accumulator parts"
    ]
})


### 5 · Embeddings & similarity for HS mapping


In [7]:
prod_emb = embedder.embed_documents(products)
hs_emb   = embedder.embed_documents(hs6_sample.description.tolist())

sim_hs = cosine_similarity(hs_emb, prod_emb)   # (20×10)
best = sim_hs.argmax(axis=0)
hs_best = hs6_sample.iloc[best].reset_index(drop=True)

df_hs_map = pd.DataFrame({
    "llm_product": products,
    "hs6_code": hs_best.code,
    "hs6_desc": hs_best.description,
    "similarity": sim_hs[best, range(sim_hs.shape[1])]
})
df_hs_map.style.background_gradient(cmap="Blues", subset=["similarity"])


,llm_product,hs6_code,hs6_desc,similarity
0,Ultra-clear tempered glass sheets,700719,"Safety glass, tempered or laminated",0.585421
1,Silver conductive paste,710692,"Silver, in semi-manufactured forms",0.388863
2,EVA encapsulant film,392113,EVA (ethylene-vinyl acetate) sheets,0.696485
3,Backsheet made of PVF,392010,Polyethylene sheets < 0.125 mm,0.547397
4,Aluminium alloy frame sections,760429,Aluminium alloy hollow profiles,0.745664
5,Junction boxes with bypass diodes,853710,Switchgear for voltage ≤ 1000 V,0.375000
6,Poly-silicon wafers,381800,Silicon doped for electronic use,0.553278
7,Copper ribbon inter-connects,854449,Winding wire of copper,0.516701
8,Phosphorus diffusion chemicals,282720,Calcium hydrogen-phosphate,0.442545
9,Anti-reflective coating materials,760611,Aluminium plates > 0.2 mm,0.368284


### Extension ideas

1. **Caching** – embed the full HS-06 (~5 000 codes) once, save as a `.pkl`.
2. **Thresholding** – drop matches below cosine 0.25, route for manual review.
3. **Hierarchical search** – (4 → 6 → 8 digit) pick the best 4-digit section
   first to prune the candidate set.
4. **Schema alignment** – write the mapping back into the supply-chain JSON so
   each LLM-generated product now carries an official HS-6 tag + similarity score.
5. **Bulk mode** – for thousands of products compute similarity via matrix
   multiplication on the GPU (e.g. `faiss`).
